<a href="https://colab.research.google.com/github/nmamatsashvili/Similarity-Seach-Notebooks/blob/master/OpenAI_Similarity_Search_SwAV_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1fa0b1fa-5f86-fb2b-5890-b966a2734aa1)


In [ ]:
import torch
# or WRN-101-2
model = torch.hub.load('facebookresearch/swav:main', 'resnet50')
model.eval()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
print("insert data")
import os
import time

lstResultSimilarity = []
stmtIds = []
stmtIdsUnvisited = []
start = time.perf_counter()
dir = "/content/drive/MyDrive/Colab Notebooks/data/responses_test_5"
dataFile = "WideResNet.txt"
FileExists = os.path.exists(dataFile)

if FileExists :
    file = open(dataFile,"r+")
    file.truncate()
    file.close()

insert data


In [14]:
print("get all statement IDs")
stmtIds = []
for folder in os.listdir(dir):
    stmtIds.append(folder)
stmtIdsUnvisited = list(stmtIds)
openFile = open(dataFile, "a")
end = time.perf_counter()
print("time before iteration: " + str( round(end - start, 2)) + " seconds")
print(str(len(stmtIds)) + " statements")
start = time.perf_counter()

get all statement IDs
time before iteration: 3.39 seconds
5 statements


In [ ]:
pip install -U sentence-transformers

In [15]:
# sample execution (requires torchvision)
from PIL import Image
import torch.nn as nn
from torchvision import transforms
from sentence_transformers import SentenceTransformer, util
def calculateCosineSimilarity(imagePath1, imagePath2):
  input_image = Image.open(imagePath1) 
  input_image2 = Image.open(imagePath2)
  preprocess = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  input_tensor = preprocess(input_image)
  input_tensor2 = preprocess(input_image2)
  input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
  input_batch2 = input_tensor2.unsqueeze(0) # create a mini-batch as expected by the model

  # move the input and model to GPU for speed if available
  if torch.cuda.is_available():
      
      input_batch = input_batch.to('cuda')
      input_batch2 = input_batch2.to('cuda')
      model.to('cuda')

  # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
  with torch.no_grad():
      output = model(input_batch)
      output2 = model(input_batch2)

  return util.cos_sim(output, output2)
  



In [ ]:
print("iterate over each statement pictures to find similarity")
from PIL import Image
import os
import time
from sentence_transformers import SentenceTransformer, util


cnt = 0
for stmt in stmtIds:
    stmtIdsUnvisited.remove(stmt)
    lstImagesCurrent = []
    
    for image in os.listdir(dir + "/" + stmt):
        if os.fsdecode(image).endswith(".jpg") == False:
            continue
        lstImagesCurrent.append(Image.open(f"{dir}/{stmt}/{image}").convert('RGB'))

    
    for img_lst in lstImagesCurrent:
        
        for stmtNext in stmtIdsUnvisited:
            lstImagesNext = []
            for imageNext in os.listdir(dir + "/" + stmtNext):
                if os.fsdecode(imageNext).endswith(".jpg") == False:
                    continue
                lstImagesNext.append(Image.open(f"{dir}/{stmtNext}/{imageNext}").convert('RGB')) 
                rst = calculateCosineSimilarity(f"{dir}/{stmt}/{image}", f"{dir}/{stmtNext}/{imageNext}")
                print("dir1: " + f"{dir}/{stmt}/{image}" + " ----- dir2: " + f"{dir}/{stmtNext}/{imageNext}" + " RST:" + str(rst))
                #result = f"statements: {stmt} vs {stmtNext} - images: {img_lst} vs {imageNext}" 
                #lstResultSimilarity.append({"id1":stmt, "id2":stmtNext, "img1":img_lst, "img2":imageNext})
                #openFile.write("\n" + result)
                cnt +=1
            
                
                

    print("success statement " + str(stmt))
print("end iteration " + str(cnt))

In [16]:
rst = calculateCosineSimilarity("/content/drive/MyDrive/Colab Notebooks/data/my_data/cc1.jpg", "/content/drive/MyDrive/Colab Notebooks/data/my_data/cc2.jpg")
print(rst)

tensor([[0.8735]])
